# Julia for R-Lovers
## Demo: Sleepstudy LMM

In [ ]:
using RCall;
using MixedModels;
using StatsBase, CSV, DataFrames;

R"""
library(tidyverse)
library(lme4) #package for doing linear mixed effects models in R
"""

### Sleep study data
- Dataset included in lme4 in R and MixedModels in Julia
- 18 participants restricted to 3 hours of sleep every night for 9 nights
- DV: average reaction time speed

- http://lme4.r-forge.r-project.org/slides/2011-01-11-Madison/2Longitudinal.pdf

### LMMs
- linear mixed effects models, add to linear regression the ability to account for random variance in repeated-measures designs (i.e., same participants or same items)
- technique well-used in psychology, cognitive science, linguistics, etc. 
- lme4 models often fail to converge in R (i.e. don't find a solution), requiring simplified model specification 
- takes a long time even when it does converge

## 1) Load data in Julia

In [ ]:
sleep = DataFrame(MixedModels.dataset(:sleepstudy));

names(sleep)

In [ ]:
first(sleep, 10)

In [ ]:
summarystats(sleep.reaction)

## 2) Wrangling in R

In [ ]:
@rput sleep;

In [ ]:
R"""
sleep %>% 
  group_by(days) %>%
  summarize(mean(reaction))
"""

In [ ]:
R"""
sleep %>% 
  group_by(subj) %>%
  summarize(mean(reaction))
"""

In [ ]:
R"""
ggplot(sleep, aes(x= days, y = reaction)) +
  geom_smooth(method = "lm", color = "grey", se = F) +
  geom_point(aes(color = subj), position = "dodge") + 
  theme_minimal()
"""

In [ ]:
R"""
ggplot(sleep, aes(x= days, y = reaction)) +
  geom_smooth(method = "lm", color = "grey", se = F) +
  geom_point(aes(color = as.factor(days)), position = "dodge", show.legend = FALSE) +
  facet_wrap(~subj) + 
  theme_minimal()
"""

## 3) Model in Julia

In [ ]:
@rget sleep #don't have to do this in this example, but would have to if you make changes to the df in R

LMM formula (similar to R)
Regression syntax
    - DV ~ predictors
Random effect term: 
    - accounts for difference by subj
    - random intercepts (y-axis location)
    - random slope 
    - (1 + predictor | subj)
    
 - In this case MixedModel syntax is similar to R:
     - lmer(reaction ~ days + (1 + days | subj))

In [ ]:
formula_sleep = @formula (reaction ~ days + (1 + days | subj));

In [ ]:
sleep_model = fit(MixedModel, formula_sleep, sleep);

In [ ]:
show(sleep_model)

### Example
formula_maximal = @formula (DV  ~ f_1 * f_2 * f_3 * f_4 + c_1 + c_2 + c_3 + c_4 + c_5 +
               (1 + f_1 + c_1 + c_2 + c_3 + c_4 | id) + 
               (1 + c_1 + f_2 * f_3 | item_1) +
               (1 + c_1 + f_2 * f_3 | item_2));

### Coding categorical predictors
cntrsts = merge(
    Dict(:cond => EffectsCoding(base="cond_A"),
         :education => HelmertCoding(levels=["High school", "Undergraduate", "Grad school"]),
         :id => Grouping(),
         :item => Grouping())
);

sleep_model = fit(MixedModel, formula_sleep, sleep, contrasts = cntrsts);

## Visualize model output in R

In [ ]:
using JellyMe4 #companion to lme4 / MixedModels and RCall

sleep_model_R = (sleep_model, sleep)

@rput sleep_model_R

In [ ]:
R"sleep_model_R"

In [ ]:
R"""
library(performance)
check_model(sleep_model_R)
"""

In [ ]:
R"""
library(ggeffects)
plot(ggpredict(sleep_model_R, terms = "days")) 
"""

In [ ]:
R"""
library(lattice)
dotplot(ranef(sleep_model_R))
"""

## Summary
- With the R commands from RCall (R"", @rput, @rget) you can use R for visualization and wrangling but let Julia do the "heavy lifting" of modeling
- My example uses LMMs (my use case) but you could substitute that step with any modeling methodology


Things to look out for:
- missing values may be treated differently
    - easy solution: remove NAs in R in advance
- changes in packages, especially "younger" ones
    - may have to be creative with package management 
- may be less on Stack Overflow 